# SQL 4 Data Science Project
## Bhejane Online Trading Store



<a id='Context'></a>
### 1. Context

The Bhejane trading store is an online retailer specialising in Covid essential items. The store has recently been struggling with the management of its database-related inventory system. Luckily for them, you've been hired as a consultant to fix the problem. 

Its time to flex your Ninja SQL skills! 
 
**Your mission, should you choose to accept it:**

You will receive a denormalised database consisting of two tables. Your task is to transform the database such that is in third normal form (3NF). To ensure that a consistent normalisation process is followed by each student, we have attached an Entity Relationship Diagram (ERD) which is a guideline on what tables need to be produced. Once you have normalised the database, you will be required to complete an MCQ test found under the 'Predict' tab of Athena.  

Though the normalisation process, you will be guided into acknowledging client (Bhenjane Trading) use-cases for the database, and dealing with data anomalies in SQL. 

![Bhejane company logo](https://raw.githubusercontent.com/Explore-AI/Pictures/master/SQL4DS/SQL_Predict/Bhejane.png)

<div align="center" >
    Bhejane, or the Black Rhino. Image by <a href="https://commons.wikimedia.org/wiki/File:Black_Rhino_(Diceros_bicornis)_browsing_..._(46584052962).jpg">Wikimedia Commons</a>
</div>

<a id='Imports'></a>
### 2. Imports
We use the below command to install sql_magic, this is the package that will assist with SQL syntax hightlighting.
* pip install sql_magic

Remember to start each new cell with " %%read_sql "

In [3]:

import sqlite3
import csv
from sqlalchemy import create_engine
%load_ext sql_magic

# Load SQLite database
engine  = create_engine("sqlite:///bhejane.db")
%config SQL.conn_name ='engine'

<a id='Data_description'></a>
### 3. Data description

The original database consist of 2 tables. 
* Product Table
* Transaction Table

The `Product` table consists of the stock of all inventory that Bhejane has on hand currently, or has had on hand historically. Items which are in this table are able to be purchased, and a record of all sales (transactions) in 2020 is notorised in the `Transactions` table. To link the tables - the `barcode` can be used. Any item in the `Transactions` table, must therefore appear in the `Products` table. 

In [5]:
import pandas as pd
data_description = pd.read_excel('Data Description.xlsx')
data_description

,Table Name,Column Name,Desciption
0,Products,Width,Width of the product once assembled
1,Products,Length,Length of the product once assembled
2,Products,Height,Height of the product once assembled
3,Products,Barcode,The unique product identifier
4,Products,Quantity,Number of goods in stock
5,Products,Brand,Product brand name relating to product company
6,Products,NavigationPath,Navigation path to specific product
7,Products,Colour,Name default colour for the product
8,Products,StockCountry,Country where the stock was bought from
9,Products,ProductDescription,Descriptive product name


<a id='Setting_up'></a>
### 4. Setting up the database

In [7]:
conn = sqlite3.connect('bhejane.db')
cursor = conn.cursor()

#### Creating both Products and Transaction Tables

In [8]:
%%read_sql
--DO NOT EDIT THIS CELL
DROP TABLE IF EXISTS "Products";
DROP TABLE IF EXISTS "Transactions";

CREATE TABLE "Products" (
    "Width"   REAL,
    "Length"  REAL,
    "Height"  REAL,
    "Barcode" VARCHAR(150),
    "Quantity" REAL,
    "Brand" VARCHAR(150), 
    "NavigationPath" VARCHAR(150),
    "Colour" VARCHAR(150),
    "StockCountry" VARCHAR(150),
    "ProductDescription" VARCHAR(150),
    "PackType" VARCHAR(150), 
    "Volume_litre" REAL, 
    "Warranty" VARCHAR(150), 
    "Weight_kg" REAL,
    "ItemDescription" VARCHAR(150), 
    "Price" REAL
);


CREATE TABLE "Transactions" (
    "CartID" INTEGER,
    "Barcode" VARCHAR(150), 
    "Total" REAL,
    "UserName" VARCHAR(150), 
    "InvoiceDate" DATETIME
);

Query started at 08:56:20 AM E. Africa Standard Time; Query executed in 0.00 m

#### Load Data into Product and Transaction tables

We extract the data from the csv files, and load it into a sqlite database. 



In [9]:
#Load data into Product table
with open('bhejane_covid_essentials_Products.csv','r') as fin: # `with` statement available in 2.5+
    # csv.DictReader uses first line in file for column headings by default
    dr = csv.DictReader(fin) # comma is default delimiter
    to_db = [(i['Width'],i['Length'],i['Height'], i['Barcode'], i['Quantity'], i['Brand'], i['NavigationPath'], i['Colour'], i['StockCountry'], i['ProductDescription'],i['PackType'],i['Volume_litre'],i['Warranty'],i['Weight_kg'],i["ItemDescription"],i['Price']) for i in dr]

cursor.executemany("INSERT INTO Products VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?);", to_db)
conn.commit()

with open('bhejane_covid_essentials_Transactions.csv','r') as fin: # `with` statement available in 2.5+
    # csv.DictReader uses first line in file for column headings by default
    dr = csv.DictReader(fin) # comma is default delimiter
    to_db = [(i['CartID'],i['Barcode'], i['Total'], i['UserName'], i['InvoiceDate']) for i in dr]

cursor.executemany("INSERT INTO Transactions VALUES (?, ?, ?, ?, ?);", to_db)
conn.commit()

<a id='ERD'></a>
### 5. Denormalized Database Tables

<br>

<img src="https://raw.githubusercontent.com/Explore-AI/Pictures/master/SQL4DS/SQL_Predict/Denormalized_Tables.png" alt="Denormalized Tables" border="0">

### 6. Normalize the given Database tables to the 1st Normal Form (1NF)

Given the below below target ERD we create new tables such the the database conforms to the 1st Normal Form

![1st Normal Form](https://raw.githubusercontent.com/Explore-AI/Pictures/master/SQL4DS/SQL_Predict/1stNF.png)


We create all the tables before attempting to populate them with data, this will help reduce errors that might creep up due to logical dependencies.

#### 6.1 Create the tables required for the 1st Normal Form
Using the above ERD sketch we create the required tables. We label the tables as they appear in the ERD sketch

In [10]:
%%read_sql
--#Create tables required for 1NF
DROP TABLE IF EXISTS "Products_1NF";
DROP TABLE IF EXISTS "Transactions_1NF";

CREATE TABLE "Products_1NF"(
    "Barcode" VARCHAR(150),
    "NavigationPath" VARCHAR(150),
    "ItemDescription" VARCHAR(150),
    "Colour" VARCHAR(150),
    "ProductDescription" VARCHAR(150) NOT NULL,
    "Brand" VARCHAR(150),
    "Price" REAL NOT NULL,
    "Quantity" INTEGER NOT NULL,
    "PackType" VARCHAR(150),
    "Warranty" VARCHAR(150),
    "StockCountry" VARCHAR(150),
    "Weight_kg" REAL,
    "Volume_litre" REAL,
    "Length" REAL,
    "Width" REAL,
    "Height" REAL,
    PRIMARY KEY("Barcode","NavigationPath","ItemDescription")
);

CREATE TABLE "Transactions_1NF" (
    "CartID" INTEGER NOT NULL,
    "Barcode" VARCHAR(150) NOT NULL,
    "UserName" VARCHAR(150) NOT NULL,
    "InvoiceDate" DATETIME NOT NULL,
    "Total" REAL NOT NULL,
    FOREIGN KEY ("Barcode") REFERENCES "Products_1NF" ("Barcode"),
    PRIMARY KEY("CartID", "Barcode", "UserName")
);

Query started at 08:59:15 AM E. Africa Standard Time; Query executed in 0.00 m

#### 6.2 Populate the tables you have create in the above section.

Populate the tables such that the database conforms to the 1st Normal Form

In [11]:
%%read_sql
--#Populate the 1NF tables
DELETE FROM "Products_1NF";
DELETE FROM "Transactions_1NF";

INSERT INTO "Products_1NF" ("Barcode","NavigationPath","ItemDescription","Colour","ProductDescription","Brand",
                            "Price","Quantity","PackType","Warranty","StockCountry","Weight_kg",
                            "Volume_litre","Length","Width","Height")
SELECT DISTINCT
     Barcode
    ,NavigationPath
    ,ItemDescription
    ,Colour
    ,ProductDescription
    ,Brand
    ,Price
    ,Quantity
    ,PackType
    ,Warranty
    ,StockCountry
    ,Weight_kg
    ,Volume_litre
    ,Length
    ,Width
    ,Height 
FROM 
    Products;

INSERT INTO "Transactions_1NF"("CartID","Barcode","UserName","InvoiceDate","Total")
SELECT DISTINCT 
    CartID
    ,Barcode
    ,UserName
    ,InvoiceDate
    ,Total
FROM
    Transactions;
    

Query started at 08:59:28 AM E. Africa Standard Time; Query executed in 0.00 m

###  7. Convert the database into its 2nd Normal Form (2NF).

We label the tables as they appear in the ERD sketch.

#### 7.1 Entity Relationship Diagram

![2nd Normal Form](https://raw.githubusercontent.com/Explore-AI/Pictures/master/SQL4DS/SQL_Predict/2ndNF.png)

#### 7.2 2NF Requirements
To transition from 1NF to 2NF, we need all columns in all tables to not have a partial dependancy on the PK of the table. This means that any tables which have a composite key e.g. `PRIMARY KEY("Barcode","NavigationPath","ItemDescription")` cannot have columns in the table which are dependant on only `Barcode`, `NavigationPath`, or `ItemDescription`. 

In moving to 2NF we will seperate out each of these as a primary key (for their own respective tables), and any columns dependant on one of these columns but not the others, will go into that table. The only columns remaining in the tables will be fully functionally dependant on the primary key of the table. For example, 

Thereby, we remove the partial dependancy, and will be in 2NF. We are instructed to create a unique identifier for the `Transactions_2NF`. 

**Second normal form notes**

<br>$\bullet$ Notice that `Total` is only dependant on `CartID` and not on `<CartID,barcode>`. This is a partial dependancy
<br>$\bullet$ Second normal form required no *partial functional dependancy* on the key. 
<br>$\bullet$ This can be achieved in 2 ways: create a unique identifier per row (so that there is a simple primary key), or break the table up into only the columns which are fully dependant on the composite key. 
<br>$\bullet$ We are going to opt for the easier solution, and create a unique simple primary key on the `Transactions_1NF`. We can create a unique primary key by concatenating `CartID||barcode`
<br>$\bullet$ NB - this approach does make it more challenging to get into 3NF! Which would be a good argument to opt for the latter approach. 

#### 7.3 Data Anomalies

We stumble into an error when we insert the data which violates this constraint. We query this with the data owner, and we are told that there are three transactions in the database which were errors. We are informed that *all transactions in the database correspond to products are in the database*.

These entries were captured incorrectly, and must be removed. How will we deal with this? 

In [12]:
%%read_sql
select count(distinct barcode) from Transactions_1NF where barcode not in (select barcode from products_1NF)

Query started at 09:02:29 AM E. Africa Standard Time; Query executed in 0.00 m

,count(distinct barcode)
0,2


### <font color='turquoise'>Action: remove the affected entries with a `delete` query </font>

In [13]:
%%read_sql
delete from Transactions_1NF
where barcode not in (select barcode from products_1NF)

Query started at 09:02:38 AM E. Africa Standard Time; Query executed in 0.00 m

In [14]:
%%read_sql
-- Should show no entries!
select count(distinct barcode) from Transactions_1NF where barcode not in (select barcode from products_1NF)

Query started at 09:02:45 AM E. Africa Standard Time; Query executed in 0.00 m

,count(distinct barcode)
0,0


#### 7.4 FK Requirements

_Note: sqlite does not always enforce foreign key constraints (and it is even more inconsistent with magic commands to enforce FK constraints!), so it is critical that we understand the purpose of the foreign key constraint and therefore are able to enforce the FK constraint!_

To enforce data integrity, and to ensure that strange things cannot happen - we will have a FK constraint. A foreign key forces all values of the FK, to be limited to only the values which exist as entries in the PK column of the table for which the aforementioned table is a foreign key to. For example, we cannot sell a product (in our Transactions table) which we do not have in our Products table. 

We will need to work around this by removing the erroneous data from the tables before continuing, since this would violate the FK constraint between the Transactions_2NF and Products_2NF table otherwise. 

We identify the observations in the Transactions_1NF table, and remove them from there._

#### 7.5 Client Use-Case Requirements

The data owner has advised that there were several instances of incorrect data capturing. Whenever data was partially captured, it was redone, but the partially-complete entries have not been removed from the database. 

For all tables which are going to be used, consider the table utility relative to the Products (Products_2NF) table. We are tasked with removing 'useless' entries as we take the data from 1NF to 2NF. For the sake of ease-of-use for the client, we will remove all rows in the 2NF tables `PackageContents_2NF` and `Colours_2NF` which have an empty value for the attributes that the client will use them for - these are the `ItemDescription` and `Colour` columns, respectively. 

We will make inserts into the other 2NF (`PackageContents_2NF` and `Colours_2NF`) tables using a `where` clause to remove instances of the value being an empty string, or a `null` value. This will need to be considered for the `ItemDescription` and for the `Colour` when data is inserted into the corresponding tables above (i.e. only insert data into the `PackageContents_2NF` which has a value for the `ItemDescription`, and only insert data into the `Colours_2NF` table which has a value for `Colour`_

### <font color='turquoise'>Action: Investigate these anomalies for the tables mentioned </font>

In [15]:
%%read_sql 

SELECT DISTINCT 
    ItemDescription,
    PackType,
    Warranty 
FROM
    Products
WHERE ItemDescription = '';

Query started at 09:05:26 AM E. Africa Standard Time; Query executed in 0.00 m

,ItemDescription,PackType,Warranty
0,,,Limited (6 months)
1,,,Non-Returnable
2,,,Limited (12 months)
3,,,Limited (120 months)
4,,,Limited (180 months)
5,,,Limited (18 months)
6,,,Supplier (12 months)
7,,Single,Limited (6 months)
8,,,Limited (24 months)
9,,,


### Let's do it! Construct the database above in second normal form

In [16]:
%%read_sql
--#Create tables required for 2NF

DROP TABLE IF EXISTS "Products_2NF";
DROP TABLE IF EXISTS "Transactions_2NF";
DROP TABLE IF EXISTS "Navigation_2NF";
DROP TABLE IF EXISTS "PackageContents_2NF";
DROP TABLE IF EXISTS "Colours_2NF";

CREATE TABLE Products_2NF(
    "RegistryID" INTEGER NOT NULL,
    "Barcode" VARCHAR(150) NOT NULL,
    "ProductDescription" VARCHAR(150) NOT NULL,
    "Brand" VARCHAR(150),
    "Price" REAL NOT NULL,
    "Quantity" INTEGER NOT NULL,
    "StockCountry" VARCHAR(150),
    "Weight_kg" REAL,
    "Volume_litre" REAL,
    "Length" REAL,
    "Width" REAL,
    "Height" REAL,
    "PathID" INTEGER,
    "ItemID" INTEGER,
    "ColourID" INTEGER,
    PRIMARY KEY(RegistryID AUTOINCREMENT),
    CONSTRAINT fk_nav FOREIGN KEY('PathID') REFERENCES Navigation_2NF ('PathID'),
    CONSTRAINT fk_pack FOREIGN KEY('ItemID') REFERENCES PackageContents_2NF ('ItemID'),
    CONSTRAINT fk_col FOREIGN KEY('ColourID') REFERENCES Colours_2NF ('ColourID') 
);

CREATE TABLE "Transactions_2NF" (
    "CartID||Barcode||UserName" VARCHAR(150) NOT NULL,
    "CartID" INTEGER NOT NULL,
    "Barcode" VARCHAR(150) NOT NULL,
    "UserName" VARCHAR(150) NOT NULL,
    "InvoiceDate" DATETIME NOT NULL,
    "Total" REAL NOT NULL,
    PRIMARY KEY("CartID||Barcode||UserName")
    CONSTRAINT fk_trans FOREIGN KEY('Barcode') REFERENCES Products_2NF ('Barcode')
);


CREATE TABLE "Navigation_2NF"(
    "PathID" INTEGER NOT NULL,
    "NavigationPath" VARCHAR(150),
    PRIMARY KEY("PathID" AUTOINCREMENT)
);

CREATE TABLE "Colours_2NF"(
    "ColourID" INTEGER NOT NULL,
    "Colour" VARCHAR(150),
    PRIMARY KEY("ColourID" AUTOINCREMENT)
);

CREATE TABLE "PackageContents_2NF"(
    "ItemID" INTEGER NOT NULL,
    "ItemDescription" VARCHAR(150),
    "PackType" VARCHAR(150),
    "Warranty" VARCHAR(150),
    PRIMARY KEY("ItemID" AUTOINCREMENT)
);


Query started at 09:05:34 AM E. Africa Standard Time; Query executed in 0.00 m

### Let's do it! _Populate_ the database above in second normal form

Firstly, let's consider the entries which are `null` and `= ''`. And we will only insert the relevant entries. 

In [17]:
%%read_sql
-- #Populate the tables so that they conform to 2NF

DELETE FROM "Products_2NF";
DELETE FROM "Transactions_2NF";
DELETE FROM "Navigation_2NF";
DELETE FROM "PackageContents_2NF";
DELETE FROM "Colours_2NF";


INSERT INTO "Navigation_2NF"("NavigationPath")
SELECT DISTINCT 
    NavigationPath 
FROM
    Products_1NF;
    
INSERT INTO "PackageContents_2NF"("ItemDescription","PackType","Warranty")
SELECT DISTINCT 
    ItemDescription,
    PackType,
    Warranty 
FROM
    Products_1NF
WHERE ItemDescription <> ''
        ;
    
INSERT INTO "Colours_2NF"("Colour")
SELECT DISTINCT 
    Colour
FROM
    Products_1NF
WHERE Colour <> ''
        ;

Query started at 09:05:42 AM E. Africa Standard Time; Query executed in 0.00 m

A more complicated scenario below as we insert into `Products_2NF`. We will insert from the Products_1NF table. We can use LEFT JOIN, because the data in each of these tables that `Products_1NF` has a FK to, originally came from `Products_1NF`, so we are gauranteed to get matches back. Normally however, what we would get from the FK restriction is the requirement to use _parent-table_ `LEFT JOIN` _child-table_. 

In [18]:
%%read_sql 

INSERT INTO "Products_2NF" ("PathID","ItemID","ColourID","Barcode","ProductDescription","Brand","Price",
                            "Quantity","StockCountry","Weight_kg","Volume_litre",
                            "Length","Width","Height")
SELECT
    PathID,
    ItemID,
    ColourID,
    Products_1NF.Barcode,
    ProductDescription,
    Brand,
    Price,
    Quantity,
    StockCountry,
    Weight_kg, 
    Volume_litre, 
    Length,
    Width,
    Height 
FROM 
    Products_1NF
LEFT JOIN Navigation_2NF ON Products_1NF.NavigationPath = Navigation_2NF.NavigationPath 
LEFT JOIN PackageContents_2NF ON Products_1NF.ItemDescription = PackageContents_2NF.ItemDescription
LEFT JOIN Colours_2NF ON Products_1NF.Colour = Colours_2NF.Colour

Query started at 09:06:12 AM E. Africa Standard Time; Query executed in 0.00 m

Last but not least, we insert into the `Transactions_2NF`table. Recall that we created a new PK in this table, and pay attention to how we are inserting values into the table. 

In [19]:
%%read_sql 

INSERT INTO "Transactions_2NF"("CartID||Barcode||UserName","CartID","Barcode","UserName","InvoiceDate","Total")
SELECT DISTINCT 
    CartID||Transactions_1NF.Barcode||UserName
    ,CartID
    ,Transactions_1NF.Barcode
    ,UserName
    ,InvoiceDate
    ,Total
FROM
    Transactions_1NF
LEFT JOIN Products_2NF ON Transactions_1NF.Barcode = Products_2NF.Barcode ;


Query started at 09:06:19 AM E. Africa Standard Time; Query executed in 0.00 m

<a id='Target_ERD'></a>
### 8. Convert the table into its 3rd Normal Form (3NF)

We will label the tables as they appear in the ERD sketch. In the same fashion as done in 2NF, we note which tables need to have data inserted `where column <> '' and column is not null`.

<br>

![3rd Normal Form](https://raw.githubusercontent.com/Explore-AI/Pictures/master/SQL4DS/SQL_Predict/3rdNF.png)

In [20]:
%%read_sql

--#Create tables required for 2NF
DROP TABLE IF EXISTS "Transactions_3NF";
DROP TABLE IF EXISTS "Carts_3NF";
DROP TABLE IF EXISTS "Products_3NF";
DROP TABLE IF EXISTS "Users_3NF";
DROP TABLE IF EXISTS "Navigation_3NF";
DROP TABLE IF EXISTS "PackageContents_3NF";
DROP TABLE IF EXISTS "Colours_3NF";
DROP TABLE IF EXISTS "Brands_3NF";
DROP TABLE IF EXISTS "Locations_3NF";

CREATE TABLE "Products_3NF" (
    "RegistryID" INTEGER NOT NULL,
    "Barcode" VARCHAR(150) NOT NULL,
    "ProductDescription" VARCHAR(150) NOT NULL,
    "Price" REAL NOT NULL,
    "Quantity" INTEGER NOT NULL,
    "Weight_kg" REAL,
    "Volume_litre" REAL,
    "Length" REAL,
    "Width" REAL,
    "Height" REAL,
    "PathID" INTEGER,
    "ItemID" INTEGER,
    "ColourID" INTEGER,
    "BrandID" INTEGER,
    "LocationID" INTEGER,
    PRIMARY KEY("RegistryID" AUTOINCREMENT),
    FOREIGN KEY("PathID") REFERENCES "Navigation_3NF" ("PathID"),
    FOREIGN KEY("ItemID") REFERENCES "PackageContents_3NF" ("ItemID"),
    FOREIGN KEY("ColourID") REFERENCES "Colours_3NF" ("ColourID") ,
    FOREIGN KEY ("BrandID") REFERENCES "Brands_3NF" ("BrandID"),
    FOREIGN KEY ("LocationID") REFERENCES "Locations_3NF" ("LocationID")
);


CREATE TABLE "Carts_3NF" (
    "CartID" INTEGER NOT NULL,
    "InvoiceDate" DATETIME NOT NULL,
    "Total" REAL NOT NULL,
    PRIMARY KEY ("CartID")
);


CREATE TABLE "Users_3NF"(
    "UserID" INTEGER NOT NULL,
    UserName VARCHAR(50) NOT NULL,
    PRIMARY KEY("UserID" AUTOINCREMENT)
);


Query started at 09:06:49 AM E. Africa Standard Time; Query executed in 0.00 m

### <font color='turquoise'>Action: proceed with the construction of the remaining tables mentioned </font>

In [21]:
%%read_sql

CREATE TABLE "Transactions_3NF" (
    "CartID||Barcode||UserName" VARCHAR(150) NOT NULL,
    "Barcode" VARCHAR(150) NOT NULL,
    "CartID" INTEGER NOT NULL,
    "UserID" INTEGER NOT NULL,
    PRIMARY KEY("CartID||Barcode||UserName")
    FOREIGN KEY('CartID') REFERENCES "Carts_3NF" ('CartID')
    FOREIGN KEY('UserID') REFERENCES "Users_3NF" ('UserID')
);
    
CREATE TABLE "Navigation_3NF"(
    "PathID" INTEGER NOT NULL,
    "NavigationPath" VARCHAR(150),
    PRIMARY KEY("PathID")
);

CREATE TABLE "PackageContents_3NF"(
    "ItemID" INTEGER NOT NULL,
    "ItemDescription" VARCHAR(150),
    "PackType" VARCHAR(150),
    "Warranty" VARCHAR(150),
    PRIMARY KEY("ItemID")
);

CREATE TABLE "Colours_3NF"(
    "ColourID" INTEGER NOT NULL,
    "Colour" VARCHAR(150),
    PRIMARY KEY("ColourID")
);

CREATE TABLE "Brands_3NF"(
    "BrandID" INTEGER NOT NULL,
    "Brand" VARCHAR(150),
    PRIMARY KEY("BrandID" AUTOINCREMENT)
);
    
CREATE TABLE "Locations_3NF"(
    "LocationID" INTEGER NOT NULL,
    "Location" VARCHAR(150),
    "StockCountry" VARCHAR(150),
    PRIMARY KEY("LocationID" AUTOINCREMENT)
);


Query started at 09:06:53 AM E. Africa Standard Time; Query executed in 0.00 m

In [22]:
%%read_sql

--# Populate the tables to that they conform to the 3rd Normal Form
DELETE FROM "Products_3NF";
DELETE FROM "Transactions_3NF";
DELETE FROM "Carts_3NF";
DELETE FROM "Users_3NF";
DELETE FROM "Navigation_3NF";
DELETE FROM "PackageContents_3NF";
DELETE FROM "Colours_3NF";
DELETE FROM "Brands_3NF";
DELETE FROM "Locations_3NF";

INSERT INTO "Users_3NF" ("UserName")
SELECT DISTINCT UserName FROM Transactions_2NF;


Query started at 09:06:57 AM E. Africa Standard Time; Query executed in 0.00 m

### <font color='turquoise'>Action: proceed with the insertion into `Carts_3NF`.   </font>

Take note that there are duplicate values coming from `Transactions_2NF` - retain the `distinct` combinations of `CartID`,`InvoiceDate`,`Total` only.

In [23]:
%%read_sql

INSERT INTO "Carts_3NF"("CartID", "InvoiceDate","Total")
SELECT DISTINCT CartID, InvoiceDate,Total  FROM Transactions_2NF;


Query started at 09:07:04 AM E. Africa Standard Time; Query executed in 0.00 m

### <font color='turquoise'>Action: proceed with the insertion into the tables below. </font>

Similarly to the table above, we mind the occurances of duplicate entries. These can be avoided by using `SELECT distinct` to avoid inserting redundant rows. 


In [24]:
%%read_sql

INSERT INTO "Locations_3NF" ("StockCountry")
SELECT DISTINCT StockCountry  FROM Products_2NF
WHERE StockCountry <> '';

INSERT INTO "Brands_3NF" ("Brand")
SELECT DISTINCT Brand  FROM Products_2NF
WHERE Brand <> '';

INSERT INTO "Colours_3NF" ("ColourID","Colour")
SELECT DISTINCT ColourID, Colour  FROM Colours_2NF;

INSERT INTO "PackageContents_3NF" ("ItemID","ItemDescription","PackType","Warranty")
SELECT DISTINCT ItemID, ItemDescription, PackType, Warranty   FROM PackageContents_2NF;

INSERT INTO "Navigation_3NF" ("PathID","NavigationPath")
SELECT DISTINCT PathID, NavigationPath  FROM Navigation_2NF;



Query started at 09:07:18 AM E. Africa Standard Time; Query executed in 0.00 m

### <font color='turquoise'>Action: proceed with the insertion into `Products_3NF`. </font>
We take note of the joins required for additional data contained in other tables. 

In [25]:
%%read_sql

INSERT INTO "Products_3NF" ("PathID","ItemID","ColourID","BrandID","LocationID",
                            "Barcode","ProductDescription","Price",
                            "Quantity","Weight_kg","Volume_litre",
                            "Length","Width","Height")
SELECT DISTINCT
    Products_2NF.PathID,
    Products_2NF.ItemID,
    Products_2NF.ColourID,
    BrandID,
    LocationID,
    Products_2NF.Barcode,
    ProductDescription,
    Price,
    Quantity,
    Weight_kg,
    Volume_litre,
    Length,
    Width,
    Height
 FROM  Products_2NF  

LEFT JOIN Brands_3NF ON Products_2NF.Brand = Brands_3NF.Brand
LEFT JOIN Locations_3NF ON Products_2NF.StockCountry = Locations_3NF.StockCountry
LEFT JOIN PackageContents_3NF ON Products_2NF.ItemID = PackageContents_3NF.ItemID
LEFT JOIN Colours_3NF ON Products_2NF.ColourID = Colours_3NF.ColourID
LEFT JOIN Navigation_3NF ON Products_2NF.PathID = Navigation_3NF.PathID

Query started at 09:07:30 AM E. Africa Standard Time; Query executed in 0.00 m

### <font color='turquoise'>Action: proceed with the insertion into `Transactions_3NF`.   </font>

Follow previous proceedure, and checks. 

In [26]:
%%read_sql

INSERT INTO "Transactions_3NF"("CartID||Barcode||UserName","CartID","Barcode","UserID")
SELECT DISTINCT 
    Transactions_2NF.CartID||Transactions_2NF.Barcode||Transactions_2NF.UserName,
    Transactions_2NF.CartID,
    Transactions_2NF.Barcode,
    UserID
FROM
    Transactions_2NF
LEFT JOIN Products_3NF ON Transactions_2NF.Barcode = Products_3NF.Barcode 
LEFT JOIN Users_3NF ON Transactions_2NF.UserName = Users_3NF.UserName
LEFT JOIN Carts_3NF ON Transactions_2NF.InvoiceDate = Carts_3NF.InvoiceDate



Query started at 09:07:33 AM E. Africa Standard Time; Query executed in 0.00 m

### <font color='turquoise'>Action: Investigate the DB.   </font>

Note - the tables counts below are what we should arrive at. 

| Table Name | Count |
| --- | --- |
| Brands_3NF | 232 |
| Carts_3NF |  190 |
| Colours_3NF | 17 |
| Locations_3NF | 2 |
| Navigation_3NF | 396 |
| PackageContents_3NF | 600 |
| Products_3NF | 1214 |
| Transactions_3NF | 275 |
| Users_3NF | 158 |

In [27]:
%%read_sql 

-- How many entries are here in each of the tables in the database now? 
select 'Products_3NF' as table_name, count(*) from Products_3NF
UNION
select 'Transactions_3NF' as table_name, count(*) from Transactions_3NF
UNION
select 'Users_3NF' as table_name, count(*) from Users_3NF
UNION
select 'Navigation_3NF' as table_name, count(*) from Navigation_3NF
UNION
select 'PackageContents_3NF' as table_name, count(*) from PackageContents_3NF
UNION
select 'Colours_3NF' as table_name, count(*) from Colours_3NF
UNION
select 'Brands_3NF' as table_name, count(*) from Brands_3NF
UNION
select 'Locations_3NF' as table_name, count(*) from Locations_3NF
UNION
select 'Carts_3NF' as table_name, count(*) from Carts_3NF


Query started at 09:08:10 AM E. Africa Standard Time; Query executed in 0.00 m

,table_name,count(*)
0,Brands_3NF,232
1,Carts_3NF,190
2,Colours_3NF,17
3,Locations_3NF,2
4,Navigation_3NF,396
5,PackageContents_3NF,600
6,Products_3NF,1214
7,Transactions_3NF,275
8,Users_3NF,158


<a id='MCQ_questions'></a>
## 9. MCQ Questions

Having completed the normalisation of the database, we use the following cells to help us answer some questions. 

**Q1) How many unique products does the company have?**

In [28]:
%%read_sql
-- Write your query here:
SELECT COUNT(DISTINCT(Barcode))
FROM Products_3NF


Query started at 09:09:49 AM E. Africa Standard Time; Query executed in 0.00 m

,COUNT(DISTINCT(Barcode))
0,608


**The company has 608 unique products**

**Q2) How many users bought from Bhejane in April 2020?**

In [29]:
%%read_sql
-- Write your query here:
SELECT COUNT(DISTINCT(Transactions_3NF.UserID))
FROM Transactions_3NF
LEFT JOIN Carts_3NF
ON Transactions_3NF.CartID = Carts_3NF.CartID
WHERE Carts_3NF.InvoiceDate BETWEEN '2020-04-01 0:00:00' AND '2020-04-30 23:59:59'

Query started at 09:10:19 AM E. Africa Standard Time; Query executed in 0.00 m

,COUNT(DISTINCT(Transactions_3NF.UserID))
0,31


**31 users bought from Bhejane in April 2020**

**Q3) How many users bought 3 or more items that cost more than R1000?**

In [30]:
%%read_sql
-- Write your query here:
WITH Users
AS
(
SELECT Transactions_3NF.UserID,  COUNT(Transactions_3NF.Barcode) AS Items, Carts_3NF.Total AS Total
FROM Transactions_3NF
LEFT JOIN Carts_3NF
ON Transactions_3NF.CartID = Carts_3NF.CartID


GROUP BY Transactions_3NF.CartID

)

SELECT COUNT(DISTINCT(UserID))
FROM Users
WHERE Items >= 3 AND
Total > 1000



Query started at 09:10:58 AM E. Africa Standard Time; Query executed in 0.00 m

,COUNT(DISTINCT(UserID))
0,11


**11 users bought 3 or more items that cost more than R1000**

**Q4) Which user made the largest purchase on a single transaction?**

In [49]:
%%read_sql
-- Write your query here:

SELECT Transactions_3NF.Barcode, Users_3NF.UserName, MAX(Carts_3NF.Total) AS Total
FROM Transactions_3NF
LEFT JOIN Carts_3NF
ON Transactions_3NF.CartID = Carts_3NF.CartID
LEFT JOIN Users_3NF
ON Transactions_3NF.UserID = Users_3NF.UserID

GROUP BY Users_3NF.UserName


ORDER BY Total DESC
LIMIT 5


Query started at 08:36:33 PM E. Africa Standard Time; Query executed in 0.00 m

,Barcode,UserName,Total
0,HTB5ABG31E3,Mandla,10999.0
1,HTB5ABG31E3,Brigette,10999.0
2,HTB5FI3BC8C,Mzoxolo,3765.0
3,HTB5FI3BC8C,Andreas Peter,3765.0
4,617566827837,Zanele,3464.0


**User HTB5ABG31E3 made the largest purchase on a single transaction**

**Q5) How many components does the product: "5m Colour Changing RGB LED Strip Light" (MPTAL57588104) come with?**

In [50]:
%%read_sql
-- Write your query here:
SELECT COUNT(PackageContents_3NF.ItemID)
FROM Products_3NF
LEFT JOIN PackageContents_3NF
ON Products_3NF.ItemID = PackageContents_3NF.ItemID
WHERE Products_3NF.ProductDescription = '5m Colour Changing RGB LED Strip Light'

Query started at 08:36:38 PM E. Africa Standard Time; Query executed in 0.00 m

,COUNT(PackageContents_3NF.ItemID)
0,8


**The product: "5m Colour Changing RGB LED Strip Light" (MPTAL57588104) comes with 8 components**

**Q6) How many brands are available at Bhejane?**

In [58]:
%%read_sql
-- Write your query here:
SELECT COUNT((Brand))
FROM Brands_3NF

Query started at 08:39:04 PM E. Africa Standard Time; Query executed in 0.00 m

,COUNT((Brand))
0,232


**232 brands are available at Bhejane**

**Q7) What is the price of the "Verimark - Floorwiz 2in1 Mop"?**

In [59]:
%%read_sql
-- Write your query here:
SELECT ItemID, ProductDescription, Price
FROM Products_3NF
WHERE ProductDescription = "Verimark - Floorwiz 2in1 Mop"

Query started at 08:39:20 PM E. Africa Standard Time; Query executed in 0.00 m

,ItemID,ProductDescription,Price
0,236,Verimark - Floorwiz 2in1 Mop,179.0
1,237,Verimark - Floorwiz 2in1 Mop,179.0
2,238,Verimark - Floorwiz 2in1 Mop,179.0


**The price of the "Verimark - Floorwiz 2in1 Mop is 179"**

**Q8) Calculate the package volume of the "Russell Hobbs - Slow Cooker" using the given dimensions**

In [60]:
%%read_sql
-- Write your query here:
SELECT ItemID, ProductDescription, Length, Width, ROUND(Height) AS Height, Length * Width * Height AS Volume
FROM Products_3NF
WHERE ProductDescription = "Russell Hobbs - Slow Cooker"

Query started at 08:40:14 PM E. Africa Standard Time; Query executed in 0.00 m

,ItemID,ProductDescription,Length,Width,Height,Volume
0,173,Russell Hobbs - Slow Cooker,24.0,34.0,33.0,26928.0
1,174,Russell Hobbs - Slow Cooker,24.0,34.0,33.0,26928.0
2,175,Russell Hobbs - Slow Cooker,24.0,34.0,33.0,26928.0
3,176,Russell Hobbs - Slow Cooker,24.0,34.0,33.0,26928.0


**The package volume of the "Russell Hobbs - Slow Cooker" is 26928**

**Q9) Which user made the most transactions in the Year 2020?**

In [37]:
%%read_sql
-- Write your query here:
SELECT Transactions_3NF.UserID, Users_3NF.UserName, COUNT(Transactions_3NF.Barcode) AS Transactions
FROM Transactions_3NF
LEFT JOIN Carts_3NF
ON Transactions_3NF.CartID = Carts_3NF.CartID
LEFT JOIN Users_3NF
ON Transactions_3NF.UserID = Users_3NF.UserID
WHERE Carts_3NF.InvoiceDate BETWEEN '2020-01-01 0:00:00' AND '2020-12-31 23:59:59'
GROUP BY Transactions_3NF.UserID
ORDER BY Transactions DESC
LIMIT 10

Query started at 09:26:14 AM E. Africa Standard Time; Query executed in 0.00 m

,UserID,UserName,Transactions
0,9,Cornelis,9
1,31,Erlo,8
2,120,Heather,6
3,6,Zanele,6
4,64,Mandla,5
5,158,Janet,4
6,154,Neo,4
7,149,Lizanie,4
8,142,Mel,4
9,134,Daffy,4


**Q10) What is the total number of users that shop at Bhejane?**

In [66]:
%%read_sql
-- Write your query here:
SELECT COUNT(DISTINCT(Username)) 
FROM Users_3NF

Query started at 08:43:46 PM E. Africa Standard Time; Query executed in 0.00 m

,COUNT(DISTINCT(Username))
0,158


**The total number of users that shop at Bhejane is 158**

**Q11) What is the record count for the Colours_3NF Table?**

In [70]:
%%read_sql
-- Write your query here:
SELECT COUNT(DISTINCT(Colour))
FROM Colours_3NF

Query started at 08:44:25 PM E. Africa Standard Time; Query executed in 0.00 m

,COUNT(DISTINCT(Colour))
0,17


**The record count for the Colours_3NF Table is 17**

**Q12) What would the total price be if I had the following items in my cart?**
* MPTAL57588104
* 5000394203921
* 6932391917652

In [36]:
%%read_sql
-- Write your query here:
SELECT DISTINCT 
    ProductDescription, Barcode, Price, 
    SUM(Price) OVER () AS TotalPrice
FROM Products_3NF
WHERE Barcode = "MPTAL57588104" OR Barcode = "5000394203921" OR Barcode = "6932391917652"

Query started at 09:24:37 AM E. Africa Standard Time; Query executed in 0.00 m

,ProductDescription,Barcode,Price,TotalPrice
0,Duracell 2032 Speciality 3V Lithium Coin Batte...,5000394203921,35.0,3982.0
1,RGB Colourful Gaming Mouse Pad - Extra Large,6932391917652,329.0,3982.0
2,5m Colour Changing RGB LED Strip Light,MPTAL57588104,370.0,3982.0


**Q13) What is the barcode of the most sold product?**

In [72]:
%%read_sql
-- Write your query here:
SELECT Barcode, COUNT(Barcode)
FROM Transactions_3NF
GROUP BY Barcode 
ORDER BY COUNT(Barcode) DESC

Query started at 08:45:12 PM E. Africa Standard Time; Query executed in 0.00 m

,Barcode,COUNT(Barcode)
0,4015400541813,4
1,MPTAL00404857,3
2,HURACANNANOWAVE1,3
3,6009686620537,3
4,6001865825405,3
...,...,...
216,5000394020146,1
217,4902505163104,1
218,43859499182,1
219,4210201043577,1


**The barcode of the most sold product is 4015400541813**

**Q14) What are the products of Cornelis’ cart on the 2020-06-28 0:00:00?**

In [73]:
%%read_sql
-- Write your query here:
SELECT DISTINCT Products_3NF.ProductDescription
FROM Transactions_3NF
LEFT JOIN Products_3NF
ON Transactions_3NF.Barcode = Products_3NF.Barcode
LEFT JOIN Carts_3NF
ON Transactions_3NF.CartID = Carts_3NF.CartID
LEFT JOIN Users_3NF
ON Transactions_3NF.UserID = Users_3NF.UserID
WHERE Users_3NF.UserName = 'Cornelis' AND
Carts_3NF.InvoiceDate = '2020-06-28 0:00:00'
ORDER BY Carts_3NF.InvoiceDate 


Query started at 08:45:37 PM E. Africa Standard Time; Query executed in 0.00 m

,ProductDescription
0,Dettol Body Wash - Shower Gel - Original - 600ml
1,Vitality Aloe Vera Gel 100ml
2,Sparq Active Resistance Loop Band - Set of 5 w...
3,Protective Face Shield


**Q15) Which users bought locally produced, black-coloured products on odd-numbered months of the year, and what was the total cost (rounded to the nearest integer) of the carts containing these products?**

In [74]:
%%read_sql
-- Write your query here:
SELECT Transactions_3NF.Barcode, Users_3NF.UserName, Products_3NF.ColourID, Products_3NF.LocationID, substr(Carts_3NF.InvoiceDate,6,2) AS Month, Carts_3NF.Total
FROM Transactions_3NF

LEFT JOIN Carts_3NF
ON Transactions_3NF.CartID = Carts_3NF.CartID

LEFT JOIN Users_3NF
ON Transactions_3NF.UserID = Users_3NF.UserID

LEFT JOIN Products_3NF
ON Transactions_3NF.Barcode = Products_3NF.Barcode

WHERE Month IN ('01','03','05','07','09','11') AND
Products_3NF.LocationID = 1 AND
Products_3NF.ColourID = 5
GROUP BY Users_3NF.UserName

 


Query started at 08:46:23 PM E. Africa Standard Time; Query executed in 0.00 m

,Barcode,UserName,ColourID,LocationID,Month,Total
0,6925281939952,Carel,5,1,05,799.0
1,741311301174,Cindy,5,1,03,1999.0
2,6001246636682,Conrad,5,1,07,93.0
3,MPTAL00215561,DIMPHO,5,1,07,1523.0
4,5000394020146,Hester,5,1,07,189.0
5,7290012291685,Jeandre,5,1,07,179.0
6,6941428152961,Lacin,5,1,05,798.0
